In [15]:
import torch
from torch.utils.data import IterableDataset, DataLoader, Subset
from datetime import datetime as dt, timedelta
import pandas as pd
import os
import random
import numpy as np
import torch.nn as nn
from pandas import DataFrame as df
import mplfinance as mpf

# check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

seed = 42  # choose any seed you prefer
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

cuda


In [16]:
class PriceDataset(torch.utils.data.Dataset):
    def __init__(self, item, timespan, start_date_str, end_date_str):
        self.directory = f'C:/Github/PricePrediction/csvfiles/{item}'
        self.item = item
        self.timespan = timespan
        start_date = dt.strptime(start_date_str, '%Y-%m-%d').date()
        end_date = dt.strptime(end_date_str, '%Y-%m-%d').date()
        self.dates = [single_date.strftime("%Y-%m-%d") for single_date in self.daterange(start_date, end_date)]
        self.columns = [1, 4]  # Selecting open and close prices
        self.filenames = self.get_filenames()

    def daterange(self, start_date, end_date):
        for n in range(int((end_date - start_date).days) + 1):
            yield start_date + timedelta(n)

    def get_filenames(self):
        filenames = []
        for date in self.dates:
            filename = f"{self.directory}/{self.item}-{self.timespan}-{date}.csv"
            if os.path.exists(filename):
                filenames.append(filename)
        return filenames

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        filename = self.filenames[idx]
        df = pd.read_csv(filename, usecols=self.columns, header=None)
        return torch.tensor(df.values, dtype=torch.float16)  # Return open and close prices

def sliding_window_pct(batch):
    windows = []
    for tensor in batch:
        for i in range(tensor.shape[0] - 400 + 1):  # Create windows of size window_size
            window = tensor[i:i+400]
            pct_change = (window[:, 1] - window[:, 0]) * 100 / window[:, 0]
            windows.append(pct_change.half())  # Convert to float16
    return torch.stack(windows)

def sliding_window_binary(batch):
    windows = []
    for tensor in batch:
        for i in range(tensor.shape[0] - 400 + 1):  # Create windows of size window_size
            window = tensor[i:i+400]
            binary_change = (window[:, 1] > window[:, 0]).float()  # Calculate the binary change
            windows.append(binary_change.half())  # Convert to float16
    return torch.stack(windows)


In [17]:
# Create the dataset
train_dataset = PriceDataset('BTCUSDT', '1m', '2021-03-01', '2023-04-30')
train_loader_pct = DataLoader(train_dataset, batch_size=1, collate_fn=sliding_window_pct, shuffle=False, drop_last=True)
train_loader_binary = DataLoader(train_dataset, batch_size=1, collate_fn=sliding_window_binary, shuffle=False, drop_last=True)

test_dataset = PriceDataset('ETHUSDT', '1m', '2021-03-01', '2023-04-30')
test_loader_pct = DataLoader(test_dataset, batch_size=1, collate_fn=sliding_window_pct, shuffle=False, drop_last=True)
test_loader_binary = DataLoader(test_dataset, batch_size=1, collate_fn=sliding_window_binary, shuffle=False, drop_last=True)

In [18]:
# Size of the Dataset
print(f'Train dataset size: {len(train_dataset)}')
print(f'Test dataset size: {len(test_dataset)}')

# Size of the DataLoader (i.e., number of batches)
print(f'Train dataloader size: {len(train_loader_binary)}')
print(f'Test dataloader size: {len(test_loader_binary)}')

# Size of the tensor output by the Dataset
sample_tensor = train_dataset[0]
print(f'Shape of the tensor output by train_dataset: {sample_tensor.shape}')

# Size of the tensor output by the DataLoader
for batch in train_loader_binary:
    print(f'Shape of the tensor output by train_loader: {batch.shape}')
    break  # we break after the first batch

first_batch = next(iter(train_loader_binary))

first_batch = next(iter(train_loader_pct))
print(first_batch[0])


Train dataset size: 791
Test dataset size: 791
Train dataloader size: 791
Test dataloader size: 791
Shape of the tensor output by train_dataset: torch.Size([1440, 2])
Shape of the tensor output by train_loader: torch.Size([1041, 400])
tensor([ 0.2837,  0.2122, -0.4939, -0.2128,  0.0000, -0.1421,  0.0712,  0.0711,
         0.4976,  0.0000, -0.1415,  0.1416,  0.0000, -0.0707,  0.4951,  0.2815,
         0.5615,  0.6274, -0.0693,  0.0693,  0.0693, -0.2078,  0.0694, -0.2080,
         0.2781, -0.0693,  0.0000,  0.0000,  0.0000,  0.0000,  0.4854,  0.0000,
         0.0690,  0.2759, -0.0688, -0.0688, -0.0689, -0.1379,  0.0690,  0.1381,
        -0.0689,  0.0000,  0.0000, -0.1379, -0.1382,  0.0000, -0.1384,  0.0000,
        -0.0693,  0.0693,  0.0000,  0.0693,  0.2771, -0.1381,  0.1383, -0.1381,
        -0.1383,  0.0693, -0.0692,  0.0000,  0.1385, -0.1383, -0.0693,  0.0000,
        -0.1385,  0.0000,  0.0000,  0.0000,  0.0694, -0.0693,  0.0000, -0.0695,
         0.0695, -0.2084,  0.1392,  0.2085, -

In [19]:
class PriceChangePrediction(nn.Module):
    def __init__(self, input_dim=1, hidden_dim=100, output_dim=1, num_layers=4):
        super(PriceChangePrediction, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout = 0.1)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out)
        return out


In [20]:
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    for batch in train_loader:
        batch = batch.unsqueeze(-1).to(device).float()  # Adds an extra dimension and converts to float32
        inputs = batch[:, :-1, :]
        targets = batch[:, 1:, :]  # The targets are the next time-steps
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()


def evaluate(model, test_loader, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in test_loader:
            batch = batch.unsqueeze(-1).to(device).float()  # Adds an extra dimension and converts to float32
            inputs = batch[:, :-1, :]
            targets = batch[:, 1:, :]  # The targets are the next time-steps
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            total_loss += loss.item()
    return total_loss / len(test_loader)  # Return average loss




def train_and_evaluate(model, model_name, train_loader, test_loader, criterion, optimizer, epochs, device):
    best_val_loss = float(1e-4)

    for epoch in range(epochs):
        train(model, train_loader, criterion, optimizer, device)
        val_loss = evaluate(model, test_loader, criterion, device)
        print(f"Epoch {epoch+1}/{epochs} \t {model_name} \t Validation Loss: {val_loss:.5f}")

        # Save the model if the validation loss is the best we've seen so far.
        if val_loss < best_val_loss:
            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
            }, f'models/{model_name}.pth')
            best_val_loss = val_loss



In [21]:
# Create the models
percentage_model = PriceChangePrediction().to(device)  # Use float32
binary_model = PriceChangePrediction().to(device)  # Use float32

# Define the loss functions
percentage_criterion = nn.MSELoss()
binary_criterion = nn.BCEWithLogitsLoss()

# Define the optimizers
percentage_optimizer = torch.optim.Adam(percentage_model.parameters(), lr=0.01)
binary_optimizer = torch.optim.Adam(binary_model.parameters(), lr=0.01)

try: 
    # Load the saved models and optimizers
    percentage_checkpoint = torch.load('C:/Github/PricePrediction/docker/models/percentage_model.pth')
    binary_checkpoint = torch.load('C:/Github/PricePrediction/docker/models/binary_model.pth')

    percentage_model.load_state_dict(percentage_checkpoint['model_state_dict'])
    binary_model.load_state_dict(binary_checkpoint['model_state_dict'])

    percentage_optimizer.load_state_dict(percentage_checkpoint['optimizer_state_dict'])
    binary_optimizer.load_state_dict(binary_checkpoint['optimizer_state_dict'])
except:
    pass

# Set the number of epochs
epochs = 1

# Train and evaluate PriceChangePrediction model
train_and_evaluate(percentage_model, 'percentage_model', train_loader_pct, test_loader_pct, percentage_criterion, percentage_optimizer, epochs, device=device)

# Train and evaluate PriceDirectionPrediction model
train_and_evaluate(binary_model, 'binary_model', train_loader_binary, test_loader_binary, binary_criterion, binary_optimizer, epochs, device=device)


OutOfMemoryError: CUDA out of memory. Tried to allocate 3.57 GiB (GPU 0; 8.00 GiB total capacity; 3.45 GiB already allocated; 898.11 MiB free; 4.99 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
print("Model's state_dict:")
for param_tensor in percentage_model.state_dict():
    print(param_tensor, "\t", percentage_model.state_dict()[param_tensor].size())

print("Optimizer's state_dict:")
for var_name in percentage_optimizer.state_dict():
    print(var_name, "\t", percentage_optimizer.state_dict()[var_name])


Epoch 1/1 | percentage_model Validation Loss: 0.0161349802
Epoch 1/1 | binary_model Validation Loss: 0.6933783825

In [ ]:
random_input = torch.randn(1, 100, 1).to(device)  # Create a random tensor of shape (1, 100, 1)
random_output = model1(random_input)  # Get the output

print(random_output)

In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()